In [1]:
import pandas as pd
import numpy as np

In [6]:
HOBO_low = pd.read_csv("HOBO\Group3-Low.csv", encoding="windows-1252")
HOBO_mid = pd.read_csv("HOBO\Group3-Mid.csv", encoding="windows-1252")
HOBO_top = pd.read_csv("HOBO\Group3-Top.csv", encoding="windows-1252")
CUBE = CUBE = pd.read_excel("CUBE\Group 3-Cube.xlsx")

In [ ]:
v_a_low = HOBO_low.iloc[:, 3]
v_a_mid = HOBO_mid.iloc[:, 3]
v_a_top = HOBO_top.iloc[:, 3]

In [ ]:
def draught_rate_comfort(v_ave, t_op, interval):
    DR_standards = np.zeros(len(v_ave), 3)
    for i in range(len(v_ave-interval)):

        v_sum_squared = 0
        v_rms = 0
        for j in range(interval):
            v_sum_squared += v_ave[i+j]*v_ave[i+j]
        v_rms = ((1/interval)*v_sum_squared)**0.5
        for j in range(interval):
            TU = v_rms/v_ave[i+j]
            DR_standards[0,i+j] = (34 - t_op[i+j])*((v_ave[i+j]-0.05)**0.62)*(0.37*v_ave[i+j]*TU+3.14)
        i = i + interval

        for k in range(len(v_ave)):
            if DR_standards[0,k] <= 10:
                DR_standards[2,k] = "Cat I"
            elif DR_standards[0,k] <= 20:
                DR_standards[2,k] = "Cat II"
            elif DR_standards[0,k] <= 30:
                DR_standards[2,k] = "Cat III"
            else:
                DR_standards[2,k] = "X"

        for k in range(len(v_ave)):
            if DR_standards[0,k] <= 20:
                DR_standards[1,k] = "ok"
            else:
                DR_standards[1,k] = "X"

    return DR_standards

    

In [ ]:
Tpr_north = CUBE.iloc[:,4]
Tpr_south = CUBE.iloc[:,6]
Tpr_east = CUBE.iloc[:,5]
Tpr_west = CUBE.iloc[:,7]
Tpr_up = CUBE.iloc[:,3]
Tpr_down = CUBE.iloc[:,8]

In [ ]:
def radiant_temperature_asymmetry_comfort(n, s, e, w, u, d):
    DT_ns = []
    DT_ew = []
    DT_ud = []
    for i in range(len(n)):
        DT_ns.append(n[i] - s[i])
        DT_ew.append(e[i] - w[i])
        DT_ud.append(u[i] - d[i])
    